In [2]:
import numpy as np
import pandas as pd
import torch
from torch import nn
# import glob
# import seaborn as sn  # for heatmaps
# from sklearn.metrics import confusion_matrix
# import matplotlib.pyplot as plt
# %matplotlib inline
# import glob

# Pending

In [15]:
def import_sepctrograms(path,columns=columns):
    data = dict()
    for label in os.listdir(path):
        data[label] = dict()
        for f in listdir(path+'/'+label):
            user = f.split('.')[0].split('_')[1]
            df = pd.read_csv(path+'/'+label+'/'+f, names=columns)
            if user not in data[label]:
                data[label][user] = []
            data[label][user].append(df)
        print(label)
    return data

# path = "E:/external_data/Experiment2/spectrogram_data_by_activity_csv"
# data = import_sepctrograms(path)

In [9]:
columns = [f"col_{i+1}" for i in range(501)]
window_size=65
slide_size=30
dirc = 'E://external_data/Experiment4/Spectrogram_data_csv_files/CSI_data'

X,y  = import_data(dirc,columns=columns,window_size=window_size,slide_size=slide_size) # 

lay
picking
sit
walking
waving
stand_l
stand_s


In [ ]:


def cross_validation(item_func,X,y,n_splits):

    acc_score = []

    kf = KFold(n_splits)
    
    for train_index , test_index in kf.split(X):
        X_train , X_test = X[train_index], X[test_index]
        y_train , y_test = y[train_index], y[test_index]
        train_loader, test_loader = create_dataloader(X_train, X_test, y_train, y_test)
        
        # create model 
        model,optimizer,criterion = item_func()
        
        # train model
        model.fit(X_train,y_train)
        pred_values = model.predict(X_test)
        
        # evaluate
        acc = accuracy_score(pred_values , y_test)
        acc_score.append(acc)

    avg_acc_score = sum(acc_score)/k

    print('accuracy of each fold - {}'.format(acc_score))
    print('Avg accuracy : {}'.format(avg_acc_score))
    
    return 

In [56]:
def contrastive_loss(p1,p2,t=1):
    val = p1.dot(p2.transpose())/t
    val = np.exp(val)
    val = -np.log(np.diag(val)/np.sum(val,axis=1))
    return val

p1 = np.array([
 [ 0.70374682, -0.18682394, -0.68544673],
 [ 0.15465702,  0.32303224,  0.93366556],
 [ 0.53043332, -0.83523217, -0.14500935],
 [ 0.68285685, -0.73054075,  0.00409143],
 [ 0.76652431,  0.61500886,  0.18494479]])

p2 = p1 + 0.1*np.random.random(p1.shape)

# contrastive_loss(p1,p2,0.1)



In [86]:


def contrastive_loss(z1,z2,t=1):
    """
    One to N
    """
    a = torch.mm(z1,z2.transpose(0,1))
    b = torch.norm(z1,dim=1)*torch.norm(z2,dim=1)
    sim_mat = torch.div(a,b)
    mask = torch.ones_like(sim_mat,dtype=bool).fill_diagonal_(0)
    pos = torch.diag(sim_mat).reshape(-1,1)
    neg = sim_mat[mask].reshape(pos.shape[0],-1)
    logits = torch.cat((pos, neg), dim=1)
    labels = torch.zeros(pos.shape[0]).long()
    loss = torch.nn.CrossEntropyLoss()(logits,labels)
    return loss

z1 = torch.tensor(p1)
z2 = torch.tensor(p2)
z3 = torch.zeros_like(z1)

In [128]:

# z = torch.cat((z1,z2))
# a = torch.mm(z,z.transpose(0,1))
# b = torch.norm(z,dim=1)*torch.norm(z,dim=1)
# sim_mat = torch.div(a,b)

In [143]:
print((0,1,1,2,2))
print(z1,'\n',z2)
torch.mm(z1,z2.transpose(0,1))

(0, 1, 1, 2, 2)
tensor([[ 0.7037, -0.1868, -0.6854],
        [ 0.1547,  0.3230,  0.9337],
        [ 0.5304, -0.8352, -0.1450],
        [ 0.6829, -0.7305,  0.0041],
        [ 0.7665,  0.6150,  0.1849]], dtype=torch.float64) 
 tensor([[ 0.7574, -0.1747, -0.6278],
        [ 0.2005,  0.3507,  1.0219],
        [ 0.5990, -0.7372, -0.1021],
        [ 0.7229, -0.6615,  0.0377],
        [ 0.8089,  0.6713,  0.2799]], dtype=torch.float64)


tensor([[ 0.9960, -0.6249,  0.6292,  0.6065,  0.2519],
        [-0.5255,  1.0984, -0.2408, -0.0667,  0.6033],
        [ 0.6387, -0.3347,  0.9482,  0.9305, -0.1722],
        [ 0.6423, -0.1151,  0.9471,  0.9771,  0.0631],
        [ 0.3570,  0.5583, -0.0131,  0.1543,  1.0846]], dtype=torch.float64)

5

In [87]:
from losses import SupConLoss

In [265]:
from sklearn.preprocessing import OneHotEncoder

lb = np.array([1,2,1,2,1])
# lb =  OneHotEncoder().fit_transform(lb.reshape(-1,1)).toarray()
print(lb)
lb = torch.tensor(lb)

pair1 = torch.cat((z1.unsqueeze(1),z1.unsqueeze(1)),dim=1)
pair2 = torch.cat((z1.unsqueeze(1),z2.unsqueeze(1)),dim=1)
pair3 = torch.cat((z3.unsqueeze(1),z3.unsqueeze(1)),dim=1)
pair4 = torch.cat((z1.unsqueeze(1),z3.unsqueeze(1)),dim=1)

[1 2 1 2 1]


In [269]:
supcon = SupConLoss(temperature=0.01,stack=True)
# z1 = nn.functional.normalize(z1,dim=1)
supcon.forward(z2,labels=lb)

tensor(9.3098, dtype=torch.float64)

In [192]:
class SupConLoss(nn.Module):
    """


    Supervised Contrastive Learning: https://arxiv.org/pdf/2004.11362.pdf.
    It also supports the unsupervised contrastive loss in SimCLR

    origin: https://github.com/HobbitLong/SupContrast.git
    """

    def __init__(self, temperature=0.07, contrast_mode='all',
                 base_temperature=0.07,stack=True):
        super(SupConLoss, self).__init__()
        self.temperature = temperature
        self.contrast_mode = contrast_mode
        self.base_temperature = base_temperature
        self.stack = stack

    def forward(self, features, labels=None, mask=None):
        """Compute loss for model. If both `labels` and `mask` are None,
        it degenerates to SimCLR unsupervised loss:
        https://arxiv.org/pdf/2002.05709.pdf
        Args:
            features: hidden vector of shape [bsz, n_views, ...].
            labels: ground truth of shape [bsz].
            mask: contrastive mask of shape [bsz, bsz], mask_{i,j}=1 if sample j
                has the same class as sample i. Can be asymmetric.
        Returns:
            A loss scalar.
        """
        device = (torch.device('cuda')
                  if features.is_cuda
                  else torch.device('cpu'))
        
        if self.stack == True:
            features = features.unsqueeze(1)
            features = torch.cat((features,features),dim=1)
            
        

        if len(features.shape) < 3:
            raise ValueError('`features` needs to be [bsz, n_views, ...],'
                             'at least 3 dimensions are required')
        if len(features.shape) > 3:
            features = features.view(features.shape[0], features.shape[1], -1)

        batch_size = features.shape[0]
        if labels is not None and mask is not None:
            raise ValueError('Cannot define both `labels` and `mask`')
        elif labels is None and mask is None:
            mask = torch.eye(batch_size, dtype=torch.float32).to(device)
        elif labels is not None:
            labels = labels.contiguous().view(-1, 1)
            if labels.shape[0] != batch_size:
                raise ValueError('Num of labels does not match num of features')
            mask = torch.eq(labels, labels.T).float().to(device)
        else:
            mask = mask.float().to(device)

        contrast_count = features.shape[1]
        contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)
        if self.contrast_mode == 'one':
            anchor_feature = features[:, 0]
            anchor_count = 1
        elif self.contrast_mode == 'all':
            anchor_feature = contrast_feature
            anchor_count = contrast_count
        else:
            raise ValueError('Unknown mode: {}'.format(self.contrast_mode))

        # compute logits
        anchor_dot_contrast = torch.div(
            torch.matmul(anchor_feature, contrast_feature.T),
            self.temperature)
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
        logits = anchor_dot_contrast - logits_max.detach()

        # tile mask
        mask = mask.repeat(anchor_count, contrast_count)
        # mask-out self-contrast cases
        logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * anchor_count).view(-1, 1).to(device),
            0
        )
        mask = mask * logits_mask

        # compute log_prob
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))

        # compute mean of log-likelihood over positive
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)

        # loss
        loss = - (self.temperature / self.base_temperature) * mean_log_prob_pos
        loss = loss.view(anchor_count, batch_size).mean()

        return loss